# Assignment
This project aims to use AutoML to predict clinical outcomes using imaging and clinical variables. The imaging modality of interest is positron emission tomography (PET). We provide a data dictionary sheet that has an explanation for what each variable resembles. 

The outcome of interest to predict is MACE with heart failure (`o_mace` column). Variables to include in the model are highlighted in <font color='gold'>yellow </font>. (This includes the bare minimum of variables, avoiding known collinearity). Variables highlighted in <font color = 'blue'>blue</font> are variables that are known to be collinear, with some of the yellow-highlighted ones. 

As a **second step**, if you can please build another predictive model **using BOTH the <font color = 'gold'>yellow</font> and the <font color = 'blue'>blue</font> highlighted variables**, just to assess if some of the collinear variables may be more predictive than others. (This model will have a large number of variables in the model). 

Variables **not to be used** for building the model (may be used at a later point) are highlighted in <font color = 'grey'>grey</font>. 

One thing to note is the unindexed values for `lvmass`, `EF`, and `systolic/diastolic` volumes but not the indexed ones (indexed ones end with `_ind`). This is to avoid collinearity, too, so only include the indexed ones with the rest of the variables in the model for model 2 but not model 1. 
 
Once the model is built, we need to explain the outcome based on different groups of interest (i.e, different age groups, different race/ethnicity groups ). For interpretability, we can explore different options; LIME, SHAP,..etc.

# Setup

In [1]:
# Install the packages defined in `requirements.txt`
# !pip install -r requirements.txt

In [1]:
# Working with data frames
import pandas as pd
import numpy as np
import sklearn.datasets

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from tpot import TPOTClassifier

# Misc
import pickle
import datetime
import warnings
#from google.colab.output import eval_js
#eval_js('google.colab.output.setIframeHeight("500")') # output cell size in GC
warnings.filterwarnings('ignore') # disable warnings

C:\Users\burno\AppData\Local\Programs\Python\Python38\lib\site-packages\tpot\builtins\__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")


# Data Import
Because the data are confidential, we are not allowed to include it in the repository. Hence, we import it from a local machine outside of the working directory (the present repository).

Of note, we mport two sheets from the Excel workbook: the data and the legend (data dictionary) sheets. The latter is used for, e.g., excluding the <font color='grey'> grey </font> features, as well as building models without and with <font color='blue'> blue </font> features.

In [2]:
df_raw = pd.read_excel('df_project2.xlsx', sheet_name = 'complete_db') # read the df from outside the directory 
df_legend = pd.read_excel('df_project2.xlsx', sheet_name = 'data_dictionary') # read the data dictionary

df_legend.head()

,position,name_col,name,type,isnumeric,format,vallab,varlab,Explanation,column_color,no_yes_vars
0,1,annon_serial_num,annon_serial_num,float,1,%9.0g,NaN,group(id pet_date),NaN,yellow,0.0
1,2,sd_age,sd_age,byte,1,%10.0g,NaN,sd_age,patient age,yellow,0.0
2,3,sd_sex,sd_sex,str6,0,%9s,NaN,sd_sex,patient gender,yellow,0.0
3,4,sd_race_unified,sd_race_unified,str38,0,%38s,NaN,sd_race_unified,patient race,yellow,0.0
4,5,sd_ethn_unified,sd_ethn_unified,str12,0,%12s,NaN,sd_ethn_unified,patient ethinicity,blue,0.0


## Basic Data Preprocessing

In [3]:
# remove non-_ind columns
print(f"df_raw dimensions before removing non-'_ind' cols:{df_raw.shape} ")
# Find columns that end with '_ind'
ind_cols = df_raw.columns[df_raw.columns.str.endswith('_ind')]
# Find column names that have both 'raw' and 'index' column
colnames_non_ind = ind_cols.str.replace('_ind', '')

# Remove the non-_ind columns
remove_yellows = []
for col in colnames_non_ind:
  all_cols = df_raw.columns[df_raw.columns.str.startswith(col)]
  for col2 in all_cols:
    if '_ind' in col2:
      pass
    else:
      remove_yellows.append(col2)
      df_raw = df_raw.drop(columns = [col2])
print(f"df_raw dimensions after removing non-'_ind' cols:{df_raw.shape} ")

# Find the column names based on color
grey_cols = df_legend[df_legend['column_color'] == 'grey']['name_col']
blue_cols = df_legend[df_legend['column_color'] == 'blue']['name_col']
yellow_cols = df_legend[df_legend['column_color'] == 'yellow']['name_col']
# Remove the non-_ind cols
yellow_cols = yellow_cols[~yellow_cols.isin(remove_yellows)].reset_index(drop = True)
green_cols = df_legend[df_legend['column_color'] == 'green']['name_col']


# Find the binary No/Yes vars (to be replaced with 0/1)
yes_no_cols = df_legend[df_legend['no_yes_vars'] == 1]['name_col'].values
df_raw[yes_no_cols] = df_raw[yes_no_cols].replace(to_replace=['No', 
                                                              'Yes'], 
                                                  value=[0, 1]) # replace No-Yes to 0-1

# Binarize 'sex' variable
df_raw['sd_sex'] = df_raw['sd_sex'].replace(to_replace=['Male', 
                                                        'Female'], 
                                            value=[0, 1])

# Binarize 'pharm_stressor'
df_raw['pharm_stressor'] = df_raw['pharm_stressor'].replace(to_replace=['Adenosine', 
                                                                        'Regadenoson'], 
                                                            value=[0, 1])

# Convert 'sd_bmi_cat' and 'cac_cat' to ordered numeric variable
df_raw['sd_bmi_cat'] = df_raw['sd_bmi_cat'].replace(to_replace = ['Underweight (<18.5kg/m2)',
                                           'Normal (18.5-25kg/m2)',
                                           'Overweight (25-30kg/m2)',
                                           'Obese (≥30kg/m2)'
                                           ],
                             value = [1,2,3,4])

df_raw['cac_cat'] = df_raw['cac_cat'].replace(to_replace = ['CACS 0', 
                                        'CACS 1-99','CACS 100-399',
                                        'CACS 400-999', 'CACS ≥1000'],
                          value = [0,1,2,3,4])

df_raw dimensions before removing non-'_ind' cols:(4323, 109) 
df_raw dimensions after removing non-'_ind' cols:(4323, 105) 


## Dataset Extraction

In [4]:
# Extract the target
target = df_raw[green_cols]['o_mace']

# Create the yellow X features df (X1)
X1 = df_raw[yellow_cols]

# Create the yellow-blue X features df (X2)
X2 = df_raw.drop(columns = grey_cols.append(green_cols))

# Drop the ID-col (it's sequential and can be easily retrieved)
X1 = X1.drop(columns = 'annon_serial_num')
X2 = X2.drop(columns = 'annon_serial_num')

# Convert `object` vars to dummies (TPOT might not deal well with cat-vars)
X1 = pd.get_dummies(X1, drop_first = True)
X2 = pd.get_dummies(X2, drop_first = True)


# Print the dimensions
print(f'Initial df dimensions: {df_raw.shape}')
print(f'Yellow X df dimensions: {X1.shape}')
print(f'Yellow-blue X df dimensions: {X2.shape}')
print(f'target dimensions: {target.shape}')

Initial df dimensions: (4323, 105)
Yellow X df dimensions: (4323, 60)
Yellow-blue X df dimensions: (4323, 84)
target dimensions: (4323,)


In [5]:
# saving data for interpretability task
#X1.to_csv('X1.csv', index=False)
#X2.to_csv('X2.csv', index=False)
#target.to_csv('target.csv', index=False)

## Check for missing values and other data characteristics

In [6]:
print('Number of missing values:')
print()
print(f'Target y: {target.isnull().sum()}')
#print(f'Target X1: {X1.isnull().sum()}')
#print(f'Target X2: {X2.isnull().sum()}')

Number of missing values:

Target y: 0


# Train-test split

In [7]:
# Train-test split for yellow X
X1_train, X1_val, y1_train, y1_val = train_test_split(X1, target, test_size=0.1, random_state=42)

# Train-test split for yellow-blue X
X2_train, X2_val, y2_train, y2_val = train_test_split(X2, target, test_size=0.1, random_state=42)

In [8]:
#models to pickle
X1_train.to_pickle("./X1_train.pkl")
y1_train.to_pickle("./y1_train.pkl")
X2_train.to_pickle("./X2_train.pkl")
y2_train.to_pickle("./y2_train.pkl")
X1_val.to_pickle("./X1_val.pkl")
y1_val.to_pickle("./y1_val.pkl")
X2_val.to_pickle("./X2_val.pkl")
y2_val.to_pickle("./y2_val.pkl")

# TPOT
- Maybe also see this tutorial: https://medium.com/analytics-vidhya/learn-to-use-tpot-an-automl-tool-4c52148c2bc9
- A paper using TPOT for medical classification: https://academic.oup.com/bioinformatics/article/36/6/1772/5614811
- More about `TPOTClassifier` class args: http://epistasislab.github.io/tpot/using/
- TPOT Github repo: https://github.com/EpistasisLab/tpot


# 1st `tpot` run

In [9]:
def build_tpot(X_train, y_train, max_time_mins = None):

  # Initialize the TPOT framework for classification.
  tpot = TPOTClassifier(
    generations=100, # Number of iterations to run the pipeline optimization process.
    population_size=20, # Number of individuals to retain in the GP population every generation. 
    scoring = 'f1_weighted', # use the wieghted F1 score
    cv = 5,
    n_jobs = -2, # all CPUs but one are used.
    max_time_mins = max_time_mins,
    max_eval_time_mins = 5,
    random_state = 42,
    verbosity = 2
  )

  # Fit the framework.
  tpot.fit(X_train, y_train)

  return tpot


def run_tpot(X, y, max_time_mins, n_splits=3):

  # Limit time to be equally between all fold and
  # also the final run on the entire dataset.
  time_limit = max(1, max_time_mins // (n_splits + 1))

  # Informative runtime information.
  start_run = datetime.datetime.now()
  estimated_end = start_run + datetime.timedelta(minutes=time_limit*(n_splits + 1))
  print(f'Start TPOT:  {start_run} Time Limit per Fold: {time_limit}min')
  print(f'Estimated end: {estimated_end}\n')

  skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
  scores = []

  # Perform startified cross validation.s
  for train_index, val_index in skf.split(X, y):

    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]

    tpot = build_tpot(X_train, y_train, time_limit)

    # Predictions and weighted F1-score.
    y_pred = tpot.predict(X_val)
    score = f1_score(y_pred, y_val, average='weighted')
    scores.append(score)

  # Building the final TPOT model on all samples.
  tpot = build_tpot(X, y, time_limit)
  avg_score = sum(scores) / len(scores)

  print()
  print(f'Weighted F1-score: {avg_score}\n')
  print(f'End of the TPOT system execution: {datetime.datetime.now()}\n')

  return tpot

In [10]:
tpot_X1 = run_tpot(X1_train, y1_train, max_time_mins=60*5)

# Save the model
tpot_X1.export('tpot_models/tpot_X1.py')

# Save the model as pickle
with open("./tpot_model_X1.pickle", "wb") as f:
    pickle.dump({"template": tpot_X1.fitted_pipeline_, "_imputed": tpot._imputed}, f)

Start TPOT:  2023-01-03 14:13:50.574965 Time Limit per Fold: 75min
Estimated end: 2023-01-03 19:13:50.574965

Imputing missing values in feature set
                                                                            
Generation 1 - Current best internal CV score: 0.9048399014158349
                                                                            
Generation 2 - Current best internal CV score: 0.9048399014158349
                                                                            
Generation 3 - Current best internal CV score: 0.9057707282764536
                                                                             
Generation 4 - Current best internal CV score: 0.9058521849826461
                                                                              
Generation 5 - Current best internal CV score: 0.9058521849826461
                                                                              
Generation 6 - Current best internal CV score: 0.90601

In [11]:
tpot_X2 = run_tpot(X2_train, y2_train, max_time_mins=60)

# Save the model
tpot_X2.export('tpot_models/tpot_X2.py')

# Save the model as pickle
with open("./tpot_model_X2.pickle", "wb") as f:
    pickle.dump({"template": tpot_X2.fitted_pipeline_, "_imputed": tpot._imputed}, f)

Start TPOT:  2023-01-03 01:55:22.770080 Time Limit per Fold: 120min
Estimated end: 2023-01-03 09:55:22.770080


120.13 minutes have elapsed. TPOT will close down.
TPOT closed during evaluation in one generation.


TPOT closed prematurely. Will use the current best pipeline.
Best pipeline: ExtraTreesClassifier(GradientBoostingClassifier(input_matrix, learning_rate=0.01, max_depth=4, max_features=0.6000000000000001, min_samples_leaf=12, min_samples_split=7, n_estimators=100, subsample=0.6000000000000001), bootstrap=True, criterion=entropy, max_features=0.8, min_samples_leaf=19, min_samples_split=5, n_estimators=100)

120.28 minutes have elapsed. TPOT will close down.
TPOT closed during evaluation in one generation.


TPOT closed prematurely. Will use the current best pipeline.
Best pipeline: GradientBoostingClassifier(SGDClassifier(VarianceThreshold(input_matrix, threshold=0.005), alpha=0.001, eta0=0.01, fit_intercept=False, l1_ratio=0.25, learning_rate=invscaling, loss=squared_hinge, pe